# 2: Dataset Setup

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, precision_recall_fscore_support
from joblib import dump, load
import pickle
import os
import pandas as pd

## Resizing images and normalizing them

In [2]:
transforms.resnet = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### Loading dataset (CIFAR-10)

In [3]:
trainset_full = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.resnet)
testset_full = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.resnet)

100%|██████████| 170M/170M [00:03<00:00, 48.7MB/s]


Selecting 500 training images and 100 test images per class


In [4]:
def get_subset(dataset, indices):
    target = np.array(dataset.targets)
    selected_indices = []
    for i in range(10):
        i_indices = np.where(target == i)[0][:indices]
        selected_indices.extend(i_indices)
    return Subset(dataset, selected_indices)

trainset = get_subset(trainset_full, 500)
testset = get_subset(testset_full, 100)

### Loading pretrained ResNet-18 and removing the last layer

In [5]:
resnet18 = models.resnet18(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(resnet18.children())[:-1])
feature_extractor.eval()

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 187MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

### Extract feature vector to get 512x1

In [6]:
def extract_features(dataset, model, batch_size=64):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    features = []
    labels = []
    with torch.no_grad():
        for images, batch_labels in dataloader:
            outputs = model(images)
            outputs = outputs.view(outputs.size(0), -1)
            features.append(outputs)
            labels.append(batch_labels)
    features = torch.cat(features).numpy()
    labels = torch.cat(labels).numpy()
    return features, labels

train_features, train_labels = extract_features(trainset, feature_extractor)
test_features, test_labels = extract_features(testset, feature_extractor)

print(train_features.shape, test_features.shape)

(5000, 512) (1000, 512)


## Using PCA to reduce the size of feature vector from 512x1 to 50x1

In [7]:
pca = PCA(n_components=50)
train_features_pca = pca.fit_transform(train_features)
test_features_pca = pca.transform(test_features)

print(train_features_pca.shape, test_features_pca.shape)

(5000, 50) (1000, 50)


Helper function to extract metrics

In [8]:
import pandas as pd

def metrics_row(y_true, y_pred, model_name):
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    return {
        "Model": model_name,
        "Accuracy": acc,
        "Macro Precision": prec,
        "Macro Recall": rec,
        "Macro F1": f1
    }

# 3: Naive Bayes

Part3.1.1 - Build confusion matrix (rows=true labels, cols=predictions)


In [9]:
import numpy as np

def confusion_matrix_np(y_true, y_pred, num_classes=10):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    cm = np.zeros((num_classes, num_classes), dtype=np.int64)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm


Part3.1.2 - Compute precision, recall, F1, and accuracy (macro + per-class)

In [10]:
def precision_recall_f1_from_cm(cm, eps=1e-12):
    tp = np.diag(cm).astype(np.float64)
    fp = cm.sum(axis=0) - tp
    fn = cm.sum(axis=1) - tp

    precision = tp / (tp + fp + eps)
    recall    = tp / (tp + fn + eps)
    f1        = 2 * precision * recall / (precision + recall + eps)

    return {
        "per_class_precision": precision,
        "per_class_recall": recall,
        "per_class_f1": f1,
        "macro_precision": precision.mean(),
        "macro_recall": recall.mean(),
        "macro_f1": f1.mean(),
        "accuracy": tp.sum() / cm.sum()
    }


Part 3.1.3 — Pretty-print evaluation report with metrics and confusion matrix

In [11]:
def print_eval_report(name, y_true, y_pred, class_names=None):
    cm = confusion_matrix_np(y_true, y_pred, num_classes=10)
    m = precision_recall_f1_from_cm(cm)

    print(f"\n===== {name} =====")
    print("Confusion Matrix (rows=true, cols=pred):")
    print(cm)
    print("\nOverall:")
    print(f"- Accuracy       : {m['accuracy']:.4f}")
    print(f"- Macro Precision: {m['macro_precision']:.4f}")
    print(f"- Macro Recall   : {m['macro_recall']:.4f}")
    print(f"- Macro F1       : {m['macro_f1']:.4f}")

    if class_names is None:
        class_names = [str(i) for i in range(10)]
    for i, cname in enumerate(class_names):
        p = m["per_class_precision"][i]
        r = m["per_class_recall"][i]
        f = m["per_class_f1"][i]
        print(f"  class {i} ({cname}): P={p:.4f} R={r:.4f} F1={f:.4f}")

cifar10_classes = [
    "airplane","automobile","bird","cat","deer",
    "dog","frog","horse","ship","truck"
]


# Part 3.2.1 — Define Gaussian Naive Bayes class (NumPy only implementation)

In [12]:
class GaussianNaiveBayes:
    """
    NumPy-only Gaussian Naive Bayes:
    - Estimate class priors, per-class means/variances
    - Predict via log-likelihood + log-prior (argmax)
    """
    def __init__(self, var_smoothing=1e-9):
        self.var_smoothing = var_smoothing
        self.classes_ = None
        self.class_priors_ = None
        self.means_ = None
        self.vars_ = None

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        C = len(self.classes_)
        N, D = X.shape

        self.means_ = np.zeros((C, D), dtype=np.float64)
        self.vars_  = np.zeros((C, D), dtype=np.float64)
        self.class_priors_ = np.zeros(C, dtype=np.float64)

        for idx, c in enumerate(self.classes_):
            Xc = X[y == c]
            self.means_[idx] = Xc.mean(axis=0)
            self.vars_[idx]  = Xc.var(axis=0) + self.var_smoothing
            self.class_priors_[idx] = len(Xc) / float(N)
        return self

    def _log_gaussian_likelihood(self, X):
        X = np.asarray(X)
        means = self.means_[None, :, :]   # (1, C, D)
        vars_ = self.vars_[None, :, :]    # (1, C, D)
        X_    = X[:, None, :]             # (N, 1, D)

        log_term  = -0.5 * (np.log(2.0 * np.pi * vars_)).sum(axis=2)
        quad_term = -0.5 * (((X_ - means) ** 2) / vars_).sum(axis=2)
        return log_term + quad_term

    def predict(self, X):
        log_like = self._log_gaussian_likelihood(X)
        log_prior = np.log(self.class_priors_)[None, :]
        scores = log_like + log_prior
        idx = np.argmax(scores, axis=1)
        return self.classes_[idx]

Part 3.2.2 — Fit method: estimate means, variances, and priors for each class

In [13]:
def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        C = len(self.classes_)
        N, D = X.shape

        self.means_ = np.zeros((C, D), dtype=np.float64)
        self.vars_  = np.zeros((C, D), dtype=np.float64)
        self.class_priors_ = np.zeros(C, dtype=np.float64)

        for idx, c in enumerate(self.classes_):
            Xc = X[y == c]
            self.means_[idx] = Xc.mean(axis=0)
            self.vars_[idx]  = Xc.var(axis=0) + self.var_smoothing
            self.class_priors_[idx] = len(Xc) / float(N)
        return self

Part 3.2.3 — Log-likelihood computation and prediction using argmax of scores

In [14]:
def _log_gaussian_likelihood(self, X):
        X = np.asarray(X)
        means = self.means_[None, :, :]   # (1, C, D)
        vars_ = self.vars_[None, :, :]    # (1, C, D)
        X_    = X[:, None, :]             # (N, 1, D)

        log_term  = -0.5 * (np.log(2.0 * np.pi * vars_)).sum(axis=2)
        quad_term = -0.5 * (((X_ - means) ** 2) / vars_).sum(axis=2)
        return log_term + quad_term

def predict(self, X):
        log_like = self._log_gaussian_likelihood(X)
        log_prior = np.log(self.class_priors_)[None, :]
        scores = log_like + log_prior
        idx = np.argmax(scores, axis=1)
        return self.classes_[idx]

Part 3.3.1 — Fit (train) the scratch GNB

In [15]:
gnb_scratch = GaussianNaiveBayes(var_smoothing=1e-9)
gnb_scratch.fit(train_features_pca, train_labels)
print("Scratch GNB fitted on PCA-50 features.")



Scratch GNB fitted on PCA-50 features.


Part 3.3.2 — Predict & print evaluation report

In [16]:
pred_scratch = gnb_scratch.predict(test_features_pca)
print_eval_report(
    "Gaussian Naive Bayes (Scratch, PCA-50)",
    test_labels, pred_scratch,
    class_names=cifar10_classes
)


===== Gaussian Naive Bayes (Scratch, PCA-50) =====
Confusion Matrix (rows=true, cols=pred):
[[80  1  0  1  0  0  1  0 13  4]
 [ 3 88  0  2  1  0  0  0  0  6]
 [ 7  0 62  8  7  4 11  0  1  0]
 [ 1  0  3 76  4 10  6  0  0  0]
 [ 1  0  5  7 74  2  3  7  1  0]
 [ 0  1  4 14  3 74  2  2  0  0]
 [ 2  0  5  7  6  0 79  1  0  0]
 [ 1  1  0  5  6  5  0 81  1  0]
 [ 7  0  1  0  1  0  0  0 88  3]
 [ 5  2  0  2  0  0  0  2  1 88]]

Overall:
- Accuracy       : 0.7900
- Macro Precision: 0.7951
- Macro Recall   : 0.7900
- Macro F1       : 0.7906
  class 0 (airplane): P=0.7477 R=0.8000 F1=0.7729
  class 1 (automobile): P=0.9462 R=0.8800 F1=0.9119
  class 2 (bird): P=0.7750 R=0.6200 F1=0.6889
  class 3 (cat): P=0.6230 R=0.7600 F1=0.6847
  class 4 (deer): P=0.7255 R=0.7400 F1=0.7327
  class 5 (dog): P=0.7789 R=0.7400 F1=0.7590
  class 6 (frog): P=0.7745 R=0.7900 F1=0.7822
  class 7 (horse): P=0.8710 R=0.8100 F1=0.8394
  class 8 (ship): P=0.8381 R=0.8800 F1=0.8585
  class 9 (truck): P=0.8713 R=0.8800 F1

Collecting metric

In [17]:
row_gnb_scratch = metrics_row(test_labels, pred_scratch, "Naive Bayes (Scratch)")

## Part 3.4.1 — Gaussian Naive Bayes (scikit-learn)

In [18]:
from sklearn.naive_bayes import GaussianNB

gnb_sklearn = GaussianNB(var_smoothing=1e-9)
gnb_sklearn.fit(train_features_pca, train_labels)
print("sklearn GNB fitted on PCA-50 features.")

sklearn GNB fitted on PCA-50 features.


Part 3.4.2 — Predict & print evaluation report

In [19]:
pred_sklearn = gnb_sklearn.predict(test_features_pca)
print_eval_report(
    "Gaussian Naive Bayes (scikit-learn, PCA-50)",
    test_labels, pred_sklearn,
    class_names=cifar10_classes
)


===== Gaussian Naive Bayes (scikit-learn, PCA-50) =====
Confusion Matrix (rows=true, cols=pred):
[[80  1  0  1  0  0  1  0 13  4]
 [ 3 88  0  2  1  0  0  0  0  6]
 [ 7  0 62  8  7  4 11  0  1  0]
 [ 1  0  3 76  4 10  6  0  0  0]
 [ 1  0  5  7 74  2  3  7  1  0]
 [ 0  1  4 14  3 74  2  2  0  0]
 [ 2  0  5  7  6  0 79  1  0  0]
 [ 1  1  0  5  6  5  0 81  1  0]
 [ 7  0  1  0  1  0  0  0 88  3]
 [ 5  2  0  2  0  0  0  2  1 88]]

Overall:
- Accuracy       : 0.7900
- Macro Precision: 0.7951
- Macro Recall   : 0.7900
- Macro F1       : 0.7906
  class 0 (airplane): P=0.7477 R=0.8000 F1=0.7729
  class 1 (automobile): P=0.9462 R=0.8800 F1=0.9119
  class 2 (bird): P=0.7750 R=0.6200 F1=0.6889
  class 3 (cat): P=0.6230 R=0.7600 F1=0.6847
  class 4 (deer): P=0.7255 R=0.7400 F1=0.7327
  class 5 (dog): P=0.7789 R=0.7400 F1=0.7590
  class 6 (frog): P=0.7745 R=0.7900 F1=0.7822
  class 7 (horse): P=0.8710 R=0.8100 F1=0.8394
  class 8 (ship): P=0.8381 R=0.8800 F1=0.8585
  class 9 (truck): P=0.8713 R=0.88

Collecting metric

In [20]:
row_gnb_sklearn = metrics_row(test_labels, pred_sklearn, " Scikit’s Gaussian Naive Bayes")

# 4: Decision Tree Implementing Gini Impurity

In [21]:
def gini_impurity(labels):
    classes, counts = np.unique(labels, return_counts=True)
    probs = counts / len(labels) #Probability
    return 1 - np.sum(probs ** 2) #Gini impurity formula

Splitting the dataset

In [22]:
def split_dataset(X, y, feature_idx, threshold): #x=features of the dataset, y=labels/targets
    left_indices = X[:, feature_idx] <= threshold
    right_indices = X[:, feature_idx] > threshold
    return X[left_indices], y[left_indices], X[right_indices], y[right_indices]

Finding the best split

In [23]:
def best_split(X, y):
    best_gini = 1
    best_feature_idx = None
    best_threshold = None

    n_features = X.shape[1] #number of features in the dataset

    for feature_idx in range(n_features):
        thresholds = np.unique(X[:, feature_idx])
        for threshold in thresholds:
            X_left, y_left, X_right, y_right = split_dataset(X, y, feature_idx, threshold)
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            gini_left = gini_impurity(y_left)
            gini_right = gini_impurity(y_right)
            gini_weight = (len(y_left) * gini_left + len(y_right) * gini_right) / len(y)

            if gini_weight < best_gini:
                best_gini = gini_weight
                best_feature_idx = feature_idx
                best_threshold = threshold

    return best_feature_idx, best_threshold

Building the decision tree

In [24]:
class DecisionTree:
    def __init__(self, depth=0, max_depth=50):
        self.max_depth = max_depth
        self.depth = depth
        self.feature_idx = None #index that is used to split
        self.threshold = None
        self.left = None #left child
        self.right = None #right child
        self.value = None

#Creates a new node for that part of the tree
def buildTree(X, y, depth=0, max_depth=50):
    node = DecisionTree(depth, max_depth)

    #stop condition (pure node or max depth reaqched)
    if len(np.unique(y)) == 1 or depth >= max_depth:
        node.value = np.bincount(y).argmax()
        return node

    #best split (picks feature and threashold with lowest Gini impurity)
    feature_idx, threshold = best_split(X, y)
    if feature_idx is None:
        node.value = np.bincount(y).argmax()
        return node

    node.feature_idx = feature_idx
    node.threshold = threshold

    X_left, y_left, X_right, y_right = split_dataset(X, y, feature_idx, threshold)
    node.left = buildTree(X_left, y_left, depth + 1, max_depth)
    node.right = buildTree(X_right, y_right, depth + 1, max_depth)

    return node

Prediction of Decision Tree

In [25]:
def predict(node, X):
    y_prediction = []
    for x in X: #start at the root node
        current = node

        #traversing the tree (until leaf is reached)
        while current.value is None:
            if x[current.feature_idx] <= current.threshold:
                current = current.left
            else:
                current = current.right
        y_prediction.append(current.value) #take the class label as prediction if leaf node is reached
    return np.array(y_prediction) #returns all the predictions

Evaluation Metrics for the Decision Tree

In [26]:
def confusionMatrix(y_true, y_prediction, num_classes=10):
    cm = np.zeros((num_classes, num_classes), dtype=int) #initializing num_class matrix to zero
    for t, p in zip(y_true, y_prediction):
        cm[t, p] += 1 #Rows = actual labels, Columns = predicted labels
    return cm

def computeMetrics(cm):
    accuracy = np.trace(cm) / np.sum(cm)
    precision = np.diag(cm) / np.maximum(cm.sum(axis=0), 1)
    recall = np.diag(cm) / np.maximum(cm.sum(axis=1), 1)
    f1 = 2 * precision * recall / np.maximum(precision + recall, 1e-6)
    return accuracy, precision, recall, f1

Training and testing

In [27]:
model_path = "decision_tree_scratch.pkl"

#Train if not saved (Saving model)
if not os.path.exists(model_path):
  tree = buildTree(train_features_pca, train_labels, max_depth=10)
  with open(model_path, "wb") as f:
    pickle.dump(tree, f)
else:
  with open(model_path, "rb") as f:
    tree = pickle.load(f)

#Evaluation
y_prediction = predict(tree, test_features_pca)

cm = confusionMatrix(test_labels, y_prediction)

accuracy, precision, recall, f1 = computeMetrics(cm)

class_names = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

print("\n===== Decision Tree (Scratch, PCA-50) =====")
print("Confusion Matrix (rows=true, cols=pred):")
print(cm)

macro_precision = np.mean(precision)
macro_recall = np.mean(recall)
macro_f1 = np.mean(f1)

print("\nOverall:")
print(f"- Accuracy       : {accuracy:.4f}")
print(f"- Macro Precision: {macro_precision:.4f}")
print(f"- Macro Recall   : {macro_recall:.4f}")
print(f"- Macro F1       : {macro_f1:.4f}")

for i, name in enumerate(class_names):
    print(f"  class {i} ({name}): "
          f"P={precision[i]:.4f} R={recall[i]:.4f} F1={f1[i]:.4f}")


===== Decision Tree (Scratch, PCA-50) =====
Confusion Matrix (rows=true, cols=pred):
[[54  7  9  6  1  1  1  1 15  5]
 [ 4 70  2  3  0  1  3  1  5 11]
 [ 4  0 42 19  9 11 11  3  1  0]
 [ 2  0  8 61  2 17  8  1  0  1]
 [ 3  0  7  9 56  4  2 18  0  1]
 [ 0  0  6 25  4 58  1  5  1  0]
 [ 4  0  6 11  2  2 71  2  2  0]
 [ 2  1  5 12 13 11  0 54  1  1]
 [18  5  2  0  0  0  2  2 64  7]
 [ 6 11  0  4  1  0  0  1  8 69]]

Overall:
- Accuracy       : 0.5990
- Macro Precision: 0.6096
- Macro Recall   : 0.5990
- Macro F1       : 0.6014
  class 0 (airplane): P=0.5567 R=0.5400 F1=0.5482
  class 1 (automobile): P=0.7447 R=0.7000 F1=0.7216
  class 2 (bird): P=0.4828 R=0.4200 F1=0.4492
  class 3 (cat): P=0.4067 R=0.6100 F1=0.4880
  class 4 (deer): P=0.6364 R=0.5600 F1=0.5957
  class 5 (dog): P=0.5524 R=0.5800 F1=0.5659
  class 6 (frog): P=0.7172 R=0.7100 F1=0.7136
  class 7 (horse): P=0.6136 R=0.5400 F1=0.5745
  class 8 (ship): P=0.6598 R=0.6400 F1=0.6497
  class 9 (truck): P=0.7263 R=0.6900 F1=0.7077

Collecting metrics

In [28]:
row_dt_scratch = metrics_row(test_labels, y_prediction, "Decision Tree (Scratch)")

Scikit-Learn Decision Tree

In [29]:
model_path = "decision_tree_sklearn.pkl"

#Train if not saved (Saving model)
if not os.path.exists(model_path):
  clf = DecisionTreeClassifier(criterion = 'gini', max_depth=10, random_state=42)
  clf.fit(train_features_pca, train_labels)
  dump(clf, model_path)
else:
  clf = load(model_path)

y_prediction_sklearn = clf.predict(test_features_pca)

#computing matrix from sklearn
cm_sklearn = confusion_matrix(test_labels, y_prediction_sklearn)
accuracy_sklearn = accuracy_score(test_labels, y_prediction_sklearn)
precision_sklearn = precision_score(test_labels, y_prediction_sklearn, average=None, zero_division=0)
recall_sklearn = recall_score(test_labels, y_prediction_sklearn, average=None, zero_division=0)
f1_sklearn = f1_score(test_labels, y_prediction_sklearn, average=None, zero_division=0)

#printing information
class_names = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

print("\n===== Decision Tree (Scikit-learn, PCA-50) =====")
print("Confusion Matrix (rows=true, cols=pred):")
print(cm_sklearn)

macro_precision = np.mean(precision_sklearn)
macro_recall = np.mean(recall_sklearn)
macro_f1 = np.mean(f1_sklearn)

print("\nOverall:")
print(f"- Accuracy       : {accuracy_sklearn:.4f}")
print(f"- Macro Precision: {macro_precision:.4f}")
print(f"- Macro Recall   : {macro_recall:.4f}")
print(f"- Macro F1       : {macro_f1:.4f}")

for i, name in enumerate(class_names):
    print(f"  class {i} ({name}): "
          f"P={precision_sklearn[i]:.4f} R={recall_sklearn[i]:.4f} F1={f1_sklearn[i]:.4f}")


===== Decision Tree (Scikit-learn, PCA-50) =====
Confusion Matrix (rows=true, cols=pred):
[[55  5  9  6  2  1  2  0 16  4]
 [ 3 69  1  4  0  1  3  1  5 13]
 [ 3  0 42 19  8 10 12  4  2  0]
 [ 3  0  8 60  1 14  8  4  1  1]
 [ 3  0  7  8 54  5  3 18  1  1]
 [ 0  0  6 25  3 58  2  5  1  0]
 [ 2  0  6 11  5  2 71  2  1  0]
 [ 1  1  4 13 13 11  0 55  1  1]
 [18  6  3  0  0  0  2  0 66  5]
 [ 5  9  0  5  1  0  0  1  7 72]]

Overall:
- Accuracy       : 0.6020
- Macro Precision: 0.6129
- Macro Recall   : 0.6020
- Macro F1       : 0.6044
  class 0 (airplane): P=0.5914 R=0.5500 F1=0.5699
  class 1 (automobile): P=0.7667 R=0.6900 F1=0.7263
  class 2 (bird): P=0.4884 R=0.4200 F1=0.4516
  class 3 (cat): P=0.3974 R=0.6000 F1=0.4781
  class 4 (deer): P=0.6207 R=0.5400 F1=0.5775
  class 5 (dog): P=0.5686 R=0.5800 F1=0.5743
  class 6 (frog): P=0.6893 R=0.7100 F1=0.6995
  class 7 (horse): P=0.6111 R=0.5500 F1=0.5789
  class 8 (ship): P=0.6535 R=0.6600 F1=0.6567
  class 9 (truck): P=0.7423 R=0.7200 F1=0

Collecting metrics

In [30]:
row_dt_sklearn = metrics_row(test_labels, y_prediction_sklearn, "Scikit’s implementation of a Decision Tree")

# 5: Multi-Layer Perceptron (MLP)

## 5.1: Define the MLP architecture

In [31]:
import torch.nn as nn
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.layer_1 = nn.Linear(input_size, 512)
        self.relu_1 = nn.ReLU()
        self.layer_2 = nn.Linear(512, 512)
        self.bn_2 = nn.BatchNorm1d(512)
        self.relu_2 = nn.ReLU()
        self.layer_3 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu_1(x)
        x = self.layer_2(x)
        x = self.bn_2(x)
        x = self.relu_2(x)
        x = self.layer_3(x)
        return x

Define Loss Function and Optimizer

In [32]:
import torch.nn as nn
import torch.optim as optim
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mlp_initial_model_path = "mlp_initial.pth"

model = MLP(input_size=train_features_pca.shape[1], num_classes=10)

if os.path.exists(mlp_initial_model_path):
    print("Loading saved MLP model...")
    model.load_state_dict(torch.load(mlp_initial_model_path, map_location=device))
    model.eval() # Set to evaluation mode after loading
    print("MLP model loaded successfully.")
else:
    print("Multilayer Perceptron (MLP) created. Model will be trained and saved.")

model.to(device) # Move model to device

criterion = nn.CrossEntropyLoss()
print("Training with PyTorch's CrossEntropyLoss")

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print("Using SGD optimizer with momentum of 0.9\n")

Multilayer Perceptron (MLP) created. Model will be trained and saved.
Training with PyTorch's CrossEntropyLoss
Using SGD optimizer with momentum of 0.9



Generate confusion matrix for initial MLP

In [33]:
import torch
import os

# Ensure mlp_initial_model_path and device are defined if running this cell independently
# (they should be defined in the previous cell)
mlp_initial_model_path = "mlp_initial.pth"

# Convert training data to PyTorch tensors
train_features_pca_tensor = torch.tensor(train_features_pca, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)

# Create a TensorDataset
train_dataset = torch.utils.data.TensorDataset(train_features_pca_tensor, train_labels_tensor)

# Define num_epochs and batch_size
num_epochs = 10
batch_size = 64

# Create a DataLoader for the training dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

if not os.path.exists(mlp_initial_model_path):
    print("Starting training for initial MLP...")
    # Training loop
    for epoch in range(num_epochs):
        model.train() # Set the model to training mode
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device) # Move data to device
            optimizer.zero_grad() # Zero the gradients
            outputs = model(inputs) # Forward pass
            loss = criterion(outputs, labels) # Calculate loss
            loss.backward() # Backpropagation
            optimizer.step() # Update weights
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")
    print("Training finished.")
    torch.save(model.state_dict(), mlp_initial_model_path)
    print(f"MLP model saved to {mlp_initial_model_path}")
else:
    print("Skipping training as MLP model already exists and was loaded.")

# Convert test data to PyTorch tensors and move to device
test_features_pca_tensor = torch.tensor(test_features_pca, dtype=torch.float32).to(device)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long).to(device)

# Set the model to evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    outputs = model(test_features_pca_tensor)
    _, predicted = torch.max(outputs.data, 1)

# Convert predictions and labels to numpy array (back to CPU for numpy conversion if on GPU)
pred_mlp_initial = predicted.cpu().numpy()

# Lines to display the results (Confusion Matrix and metrics)
print_eval_report(
    "MLP (Initial, PCA-50)",
    test_labels_tensor.cpu().numpy(), # Pass labels from CPU tensor
    pred_mlp_initial,
    class_names=cifar10_classes
)

Starting training for initial MLP...
Epoch 1/10, Loss: 1.4163
Epoch 2/10, Loss: 0.7443
Epoch 3/10, Loss: 0.6204
Epoch 4/10, Loss: 0.5415
Epoch 5/10, Loss: 0.4973
Epoch 6/10, Loss: 0.4631
Epoch 7/10, Loss: 0.4359
Epoch 8/10, Loss: 0.4118
Epoch 9/10, Loss: 0.3920
Epoch 10/10, Loss: 0.3799
Training finished.
MLP model saved to mlp_initial.pth

===== MLP (Initial, PCA-50) =====
Confusion Matrix (rows=true, cols=pred):
[[76  0  3  2  0  0  1  0 11  7]
 [ 3 92  0  1  0  0  0  0  0  4]
 [ 4  0 72  7  3  6  7  0  1  0]
 [ 0  0  4 75  2 11  6  2  0  0]
 [ 1  0  5  5 80  2  1  5  1  0]
 [ 0  0  4 14  2 75  2  2  1  0]
 [ 1  0  3  3  1  1 90  1  0  0]
 [ 2  0  0  4  7  4  0 83  0  0]
 [ 4  0  1  0  0  0  0  0 93  2]
 [ 3  4  0  2  0  0  0  0  0 91]]

Overall:
- Accuracy       : 0.8270
- Macro Precision: 0.8291
- Macro Recall   : 0.8270
- Macro F1       : 0.8271
  class 0 (airplane): P=0.8085 R=0.7600 F1=0.7835
  class 1 (automobile): P=0.9583 R=0.9200 F1=0.9388
  class 2 (bird): P=0.7826 R=0.7200

Collecting metric

In [34]:
row_mlp_initial = metrics_row(test_labels, pred_mlp_initial, "MLP")

## 5.2.1: Adding layers

In [35]:
# Define an MLP with more layers
class DeeperMLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(DeeperMLP, self).__init__()
        self.layer_1 = nn.Linear(input_size, 512)
        self.relu_1 = nn.ReLU()
        self.layer_2 = nn.Linear(512, 512)
        self.bn_2 = nn.BatchNorm1d(512)
        self.relu_2 = nn.ReLU()
        self.layer_3 = nn.Linear(512, 256) # Added a new hidden layer
        self.relu_3 = nn.ReLU()
        self.layer_4 = nn.Linear(256, num_classes) # Output layer adjusted

    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu_1(x)
        x = self.layer_2(x)
        x = self.bn_2(x)
        x = self.relu_2(x)
        x = self.layer_3(x)
        x = self.relu_3(x)
        x = self.layer_4(x)
        return x

print("Deeper Multilayer Perceptron (MLP) created")

# Instantiate the deeper MLP model
deep_mlp_model_path = "deeper_mlp.pth"
deeper_model = DeeperMLP(input_size=train_features_pca.shape[1], num_classes=10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
deeper_model.to(device)

# Define Loss Function and Optimizer for the deeper model
criterion = nn.CrossEntropyLoss()
optimizer_deeper = optim.SGD(deeper_model.parameters(), lr=0.001, momentum=0.9)

# Convert data to PyTorch tensors
train_features_pca_tensor = torch.tensor(train_features_pca, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)
test_features_pca_tensor = torch.tensor(test_features_pca, dtype=torch.float32).to(device)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long).to(device)

# Training loop
num_epochs = 10
batch_size = 64

train_dataset = torch.utils.data.TensorDataset(train_features_pca_tensor, train_labels_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

if os.path.exists(deep_mlp_model_path):
    print("Loading saved Deeper MLP model...")
    deeper_model.load_state_dict(torch.load(deep_mlp_model_path, map_location=device))
    deeper_model.eval() # Set to evaluation mode after loading
    print("Deeper MLP model loaded successfully.")
else:
    print("Starting training for Deeper MLP...")
    for epoch in range(num_epochs):
        deeper_model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer_deeper.zero_grad()
            outputs = deeper_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_deeper.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")
    print("Training finished.")
    torch.save(deeper_model.state_dict(), deep_mlp_model_path)
    print(f"Deeper MLP model saved to {deep_mlp_model_path}")

# Evaluate the deeper model
deeper_model.eval()
with torch.no_grad():
    outputs = deeper_model(test_features_pca_tensor)
    _, predicted_deeper = torch.max(outputs.data, 1)

# Generate and print evaluation report for the deeper MLP
pred_deeper_mlp = predicted_deeper.cpu().numpy()
print_eval_report(
    "MLP (Deeper, PCA-50)",
    test_labels_tensor.cpu().numpy(),
    pred_deeper_mlp,
    class_names=cifar10_classes
)

Deeper Multilayer Perceptron (MLP) created
Starting training for Deeper MLP...
Epoch 1/10, Loss: 2.0582
Epoch 2/10, Loss: 1.3901
Epoch 3/10, Loss: 0.9298
Epoch 4/10, Loss: 0.7297
Epoch 5/10, Loss: 0.6259
Epoch 6/10, Loss: 0.5655
Epoch 7/10, Loss: 0.5129
Epoch 8/10, Loss: 0.4775
Epoch 9/10, Loss: 0.4482
Epoch 10/10, Loss: 0.4202
Training finished.
Deeper MLP model saved to deeper_mlp.pth

===== MLP (Deeper, PCA-50) =====
Confusion Matrix (rows=true, cols=pred):
[[84  1  1  1  0  0  1  0  8  4]
 [ 3 92  0  1  0  0  0  0  1  3]
 [ 6  0 75  6  3  5  5  0  0  0]
 [ 1  0  4 75  1 13  5  1  0  0]
 [ 2  0  3  5 78  3  0  7  2  0]
 [ 0  0  4 14  1 76  2  2  1  0]
 [ 1  0  4  5  2  4 83  1  0  0]
 [ 2  1  0  4  9  4  0 80  0  0]
 [ 3  0  1  0  0  0  0  0 94  2]
 [ 2  3  0  1  0  0  0  0  1 93]]

Overall:
- Accuracy       : 0.8300
- Macro Precision: 0.8329
- Macro Recall   : 0.8300
- Macro F1       : 0.8306
  class 0 (airplane): P=0.8077 R=0.8400 F1=0.8235
  class 1 (automobile): P=0.9485 R=0.920

Collecting metric

In [36]:
row_mlp_deeper = metrics_row(test_labels, pred_deeper_mlp, "MLP (Deeper, PCA-50)")

## 5.2.2: Removing layers

In [37]:
# Define an MLP with fewer layers (e.g. one hidden layer)
class ShallowerMLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(ShallowerMLP, self).__init__()
        self.layer_1 = nn.Linear(input_size, 256) # Reduced hidden layer size
        self.relu_1 = nn.ReLU()
        self.layer_2 = nn.Linear(256, num_classes) # Output layer

    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu_1(x)
        x = self.layer_2(x)
        return x

print("Shallower Multilayer Perceptron (MLP) created")

# Instantiate the shallower MLP model
shallower_mlp_model_path = "shallower_mlp.pth"
shallower_model = ShallowerMLP(input_size=train_features_pca.shape[1], num_classes=10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
shallower_model.to(device)

# Define Loss Function and Optimizer for the shallower model
criterion = nn.CrossEntropyLoss()
optimizer_shallower = optim.SGD(shallower_model.parameters(), lr=0.001, momentum=0.9)

# Convert data to PyTorch tensors
train_features_pca_tensor = torch.tensor(train_features_pca, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)
test_features_pca_tensor = torch.tensor(test_features_pca, dtype=torch.float32).to(device)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long).to(device)

# Training loop
num_epochs = 10
batch_size = 64

train_dataset = torch.utils.data.TensorDataset(train_features_pca_tensor, train_labels_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

if os.path.exists(shallower_mlp_model_path):
    print("Loading saved Shallower MLP model...")
    shallower_model.load_state_dict(torch.load(shallower_mlp_model_path, map_location=device))
    shallower_model.eval() # Set to evaluation mode after loading
    print("Shallower MLP model loaded successfully.")
else:
    print("Starting training for Shallower MLP...")
    for epoch in range(num_epochs):
        shallower_model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer_shallower.zero_grad()
            outputs = shallower_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_shallower.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

    print("Training finished.")
    torch.save(shallower_model.state_dict(), shallower_mlp_model_path)
    print(f"Shallower MLP model saved to {shallower_mlp_model_path}")

# Evaluate the shallower model
shallower_model.eval()
with torch.no_grad():
    outputs = shallower_model(test_features_pca_tensor)
    _, predicted_shallower = torch.max(outputs.data, 1)

# Generate and print evaluation report for the shallower MLP
pred_shallower_mlp = predicted_shallower.cpu().numpy()
print_eval_report(
    "MLP (Shallower, PCA-50)",
    test_labels_tensor.cpu().numpy(),
    pred_shallower_mlp,
    class_names=cifar10_classes
)

Shallower Multilayer Perceptron (MLP) created
Starting training for Shallower MLP...
Epoch 1/10, Loss: 1.7531
Epoch 2/10, Loss: 1.0376
Epoch 3/10, Loss: 0.8103
Epoch 4/10, Loss: 0.7070
Epoch 5/10, Loss: 0.6437
Epoch 6/10, Loss: 0.5933
Epoch 7/10, Loss: 0.5646
Epoch 8/10, Loss: 0.5394
Epoch 9/10, Loss: 0.5291
Epoch 10/10, Loss: 0.5022
Training finished.
Shallower MLP model saved to shallower_mlp.pth

===== MLP (Shallower, PCA-50) =====
Confusion Matrix (rows=true, cols=pred):
[[81  0  2  1  0  0  0  0 11  5]
 [ 2 94  0  1  0  0  0  0  0  3]
 [ 8  0 70  3  3  6  8  1  1  0]
 [ 1  0  3 70  2 15  8  1  0  0]
 [ 2  0  3  5 77  4  0  6  1  2]
 [ 0  0  4 12  2 76  2  3  1  0]
 [ 1  0  2  2  4  2 88  0  1  0]
 [ 1  1  1  4  7  6  0 77  1  2]
 [ 7  0  2  0  0  0  0  0 88  3]
 [ 1  5  0  1  0  0  0  0  1 92]]

Overall:
- Accuracy       : 0.8130
- Macro Precision: 0.8141
- Macro Recall   : 0.8130
- Macro F1       : 0.8124
  class 0 (airplane): P=0.7788 R=0.8100 F1=0.7941
  class 1 (automobile): P

Collecting metric

In [38]:
row_mlp_shallower = metrics_row(test_labels, pred_shallower_mlp, "MLP (Shallower, PCA-50)")

## 5.3.1: Larger Hidden Layer

In [39]:
# Define an MLP with larger hidden layers (e.g. 1024 units)
class WiderMLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(WiderMLP, self).__init__()
        self.layer_1 = nn.Linear(input_size, 1024) # Increased hidden layer size
        self.relu_1 = nn.ReLU()
        self.layer_2 = nn.Linear(1024, 1024) # Increased hidden layer size
        self.bn_2 = nn.BatchNorm1d(1024)
        self.relu_2 = nn.ReLU()
        self.layer_3 = nn.Linear(1024, num_classes) # Output layer

    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu_1(x)
        x = self.layer_2(x)
        x = self.bn_2(x)
        x = self.relu_2(x)
        x = self.layer_3(x)
        return x

print("Wider Multilayer Perceptron (MLP) created")

# Instantiate the wider MLP model
wider_mlp_model_path = "wider_mlp.pth"
wider_model = WiderMLP(input_size=train_features_pca.shape[1], num_classes=10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wider_model.to(device)

# Define Loss Function and Optimizer for the wider model
criterion = nn.CrossEntropyLoss()
optimizer_wider = optim.SGD(wider_model.parameters(), lr=0.001, momentum=0.9)

# Convert data to PyTorch tensors
train_features_pca_tensor = torch.tensor(train_features_pca, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)
test_features_pca_tensor = torch.tensor(test_features_pca, dtype=torch.float32).to(device)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long).to(device)

# Training loop
num_epochs = 10
batch_size = 64

train_dataset = torch.utils.data.TensorDataset(train_features_pca_tensor, train_labels_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

if os.path.exists(wider_mlp_model_path):
    print("Loading saved Wider MLP model...")
    wider_model.load_state_dict(torch.load(wider_mlp_model_path, map_location=device))
    wider_model.eval() # Set to evaluation mode after loading
    print("Wider MLP model loaded successfully.")
else:
    print("Starting training for Wider MLP...")
    for epoch in range(num_epochs):
        wider_model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer_wider.zero_grad()
            outputs = wider_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_wider.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

    print("Training finished.")
    torch.save(wider_model.state_dict(), wider_mlp_model_path)
    print(f"Wider MLP model saved to {wider_mlp_model_path}")

# Evaluate the wider model
wider_model.eval()
with torch.no_grad():
    outputs = wider_model(test_features_pca_tensor)
    _, predicted_wider = torch.max(outputs.data, 1)

# Generate and print evaluation report for the wider MLP
pred_wider_mlp = predicted_wider.cpu().numpy()
print_eval_report(
    "MLP (Wider, PCA-50)",
    test_labels_tensor.cpu().numpy(),
    pred_wider_mlp,
    class_names=cifar10_classes
)

Wider Multilayer Perceptron (MLP) created
Starting training for Wider MLP...
Epoch 1/10, Loss: 1.1422
Epoch 2/10, Loss: 0.5851
Epoch 3/10, Loss: 0.4985
Epoch 4/10, Loss: 0.4482
Epoch 5/10, Loss: 0.4055
Epoch 6/10, Loss: 0.3967
Epoch 7/10, Loss: 0.3591
Epoch 8/10, Loss: 0.3377
Epoch 9/10, Loss: 0.3187
Epoch 10/10, Loss: 0.3096
Training finished.
Wider MLP model saved to wider_mlp.pth

===== MLP (Wider, PCA-50) =====
Confusion Matrix (rows=true, cols=pred):
[[88  0  1  1  0  0  0  0  7  3]
 [ 3 91  0  1  0  0  0  0  0  5]
 [ 6  0 71  8  5  4  6  0  0  0]
 [ 1  0  2 74  2 13  7  1  0  0]
 [ 2  0  3  7 74  2  3  8  1  0]
 [ 0  0  5 15  2 72  2  3  1  0]
 [ 1  0  3  4  2  1 88  0  1  0]
 [ 0  0  0  5  7  3  0 85  0  0]
 [ 4  0  1  0  0  0  0  0 93  2]
 [ 3  4  0  2  0  0  0  0  0 91]]

Overall:
- Accuracy       : 0.8270
- Macro Precision: 0.8303
- Macro Recall   : 0.8270
- Macro F1       : 0.8273
  class 0 (airplane): P=0.8148 R=0.8800 F1=0.8462
  class 1 (automobile): P=0.9579 R=0.9100 F1=

Collecting metric

In [40]:
row_mlp_wider = metrics_row(test_labels, pred_wider_mlp, "MLP (Wider, PCA-50)")

## 5.3.2: Smaller Hidden Layer

In [41]:
# Define an MLP with smaller hidden layers (e.g. 128 units)
class NarrowerMLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NarrowerMLP, self).__init__()
        self.layer_1 = nn.Linear(input_size, 128) # Decreased hidden layer size
        self.relu_1 = nn.ReLU()
        self.layer_2 = nn.Linear(128, 128) # Decreased hidden layer size
        self.bn_2 = nn.BatchNorm1d(128)
        self.relu_2 = nn.ReLU()
        self.layer_3 = nn.Linear(128, num_classes) # Output layer

    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu_1(x)
        x = self.layer_2(x)
        x = self.bn_2(x)
        x = self.relu_2(x)
        x = self.layer_3(x)
        return x

print("Narrower Multilayer Perceptron (MLP) created")

# Instantiate the narrower MLP model
narrower_mlp_model_path = "narrower_mlp.pth"
narrower_model = NarrowerMLP(input_size=train_features_pca.shape[1], num_classes=10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
narrower_model.to(device)

# Define Loss Function and Optimizer for the narrower model
criterion = nn.CrossEntropyLoss()
optimizer_narrower = optim.SGD(narrower_model.parameters(), lr=0.001, momentum=0.9)

# Convert data to PyTorch tensors
train_features_pca_tensor = torch.tensor(train_features_pca, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)
test_features_pca_tensor = torch.tensor(test_features_pca, dtype=torch.float32).to(device)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long).to(device)

# Training loop
num_epochs = 10
batch_size = 64

train_dataset = torch.utils.data.TensorDataset(train_features_pca_tensor, train_labels_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

if os.path.exists(narrower_mlp_model_path):
    print("Loading saved Narrower MLP model...")
    narrower_model.load_state_dict(torch.load(narrower_mlp_model_path, map_location=device))
    narrower_model.eval() # Set to evaluation mode after loading
    print("Narrower MLP model loaded successfully.")
else:
    print("Starting training for Narrower MLP...")
    for epoch in range(num_epochs):
        narrower_model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer_narrower.zero_grad()
            outputs = narrower_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_narrower.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

    print("Training finished.")
    torch.save(narrower_model.state_dict(), narrower_mlp_model_path)
    print(f"Narrower MLP model saved to {narrower_mlp_model_path}")

# Evaluate the narrower model
narrower_model.eval()
with torch.no_grad():
    outputs = narrower_model(test_features_pca_tensor)
    _, predicted_narrower = torch.max(outputs.data, 1)

# Generate and print evaluation report for the narrower MLP
pred_narrower_mlp = predicted_narrower.cpu().numpy()
print_eval_report(
    "MLP (Narrower, PCA-50)",
    test_labels_tensor.cpu().numpy(),
    pred_narrower_mlp,
    class_names=cifar10_classes
)

Narrower Multilayer Perceptron (MLP) created
Starting training for Narrower MLP...
Epoch 1/10, Loss: 1.9326
Epoch 2/10, Loss: 1.3068
Epoch 3/10, Loss: 1.0099
Epoch 4/10, Loss: 0.8434
Epoch 5/10, Loss: 0.7374
Epoch 6/10, Loss: 0.6706
Epoch 7/10, Loss: 0.6258
Epoch 8/10, Loss: 0.5840
Epoch 9/10, Loss: 0.5515
Epoch 10/10, Loss: 0.5191
Training finished.
Narrower MLP model saved to narrower_mlp.pth

===== MLP (Narrower, PCA-50) =====
Confusion Matrix (rows=true, cols=pred):
[[84  1  1  1  0  0  0  0 10  3]
 [ 3 90  0  1  0  0  0  0  2  4]
 [ 7  0 66  4  4  8  9  2  0  0]
 [ 0  0  5 69  2 13  9  1  1  0]
 [ 3  0  4  6 71  3  1 10  2  0]
 [ 0  0  4 13  1 77  2  2  1  0]
 [ 2  0  1  4  4  1 87  0  1  0]
 [ 1  0  0  3  7  6  0 82  0  1]
 [ 5  0  1  0  0  0  0  0 92  2]
 [ 2  4  0  0  0  0  0  0  2 92]]

Overall:
- Accuracy       : 0.8100
- Macro Precision: 0.8113
- Macro Recall   : 0.8100
- Macro F1       : 0.8090
  class 0 (airplane): P=0.7850 R=0.8400 F1=0.8116
  class 1 (automobile): P=0.94

Collecting metrics

In [42]:
row_mlp_narrower = metrics_row(test_labels, pred_narrower_mlp, "MLP (Narrower, PCA-50)")

# 6: Convolutional Neural Network (CNN)

## 6.1: Implementing and training a VGG11 net

Using `torch.nn.CrossEntropyLoss`, and optimize using SGD optimizer with `momentum=0.9`

Implementing VGG11 - according to assignment description

In [43]:
class VGG11(nn.Module):
  def __init__(self, num_classes=10):
    super(VGG11, self).__init__()
    #extracting image features
    self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d(2, 2)
        )
    self.classifier = nn.Sequential(
            nn.Linear(512, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )

  def forward(self, x):
    x = self.features(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x

Preparing Data for CNN (resizing to 32x32)

In [44]:
transform_cnn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


trainset_cnn = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cnn)
testset_cnn = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cnn)

trainset_cnn = get_subset(trainset_cnn, 500)
testset_cnn = get_subset(testset_cnn, 100)

trainloader = DataLoader(trainset_cnn, batch_size=64, shuffle=True)
testloader = DataLoader(testset_cnn, batch_size=64, shuffle=False)

Training the CNN

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG11(num_classes=10).to(device)

#where to store the model
model_path = "vgg11_base.pth"

#checking if model was saved and load it
if os.path.exists('vgg11_base.pth'):
  print("Loading saved model...")
  model.load_state_dict(torch.load(model_path, map_location=device))
  model.eval()
#Train from scratch
else:
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

  num_epochs = 10
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0
      for images, labels in trainloader:
          images, labels = images.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      #printing the loss to see that the model is training
      print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss/len(trainloader):.4f}")
  torch.save(model.state_dict(), model_path)

Epoch [1/10] - Loss: 2.1424
Epoch [2/10] - Loss: 1.7154
Epoch [3/10] - Loss: 1.4265
Epoch [4/10] - Loss: 1.1932
Epoch [5/10] - Loss: 0.9414
Epoch [6/10] - Loss: 0.6971
Epoch [7/10] - Loss: 0.4817
Epoch [8/10] - Loss: 0.3826
Epoch [9/10] - Loss: 0.2400
Epoch [10/10] - Loss: 0.1239


Evaluation of CNN

In [46]:
def evaluate_model(model, testloader, class_names):
    model.eval()
    device = next(model.parameters()).device
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    acc = accuracy_score(all_labels, all_preds)
    prec, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None)
    macro_prec, macro_rec, macro_f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')

    print("===== CNN (VGG11, CIFAR-10) =====")
    print("Confusion Matrix (rows=true, cols=pred):")
    print(cm)
    print("\nOverall:")
    print(f"- Accuracy       : {acc:.4f}")
    print(f"- Macro Precision: {macro_prec:.4f}")
    print(f"- Macro Recall   : {macro_rec:.4f}")
    print(f"- Macro F1       : {macro_f1:.4f}")

    for i, name in enumerate(class_names):
        print(f"  class {i} ({name}): P={prec[i]:.4f} R={rec[i]:.4f} F1={f1[i]:.4f}")
    return all_labels, all_preds

# CIFAR-10 class labels
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

labels_cnn, preds_cnn = evaluate_model(model, testloader, class_names)

#collecting metric
row_cnn = metrics_row(labels_cnn, preds_cnn, "CNN (VGG11)")


===== CNN (VGG11, CIFAR-10) =====
Confusion Matrix (rows=true, cols=pred):
[[51  4 10  5  3  1  8  1 15  2]
 [ 3 82  0  2  0  0  2  0  5  6]
 [ 9  0 48 13  6  9 10  2  2  1]
 [ 1  1 12 48  2 23  8  3  0  2]
 [ 4  2 15 14 33 10  6 12  3  1]
 [ 1  0 10 24  1 52  2  7  2  1]
 [ 0  0  7 18  2  3 69  1  0  0]
 [ 3  0  0 10  5 11  1 64  2  4]
 [ 7 10  1  2  2  1  5  0 68  4]
 [ 3 20  1  4  0  0  4  0 13 55]]

Overall:
- Accuracy       : 0.5700
- Macro Precision: 0.5852
- Macro Recall   : 0.5700
- Macro F1       : 0.5692
  class 0 (airplane): P=0.6220 R=0.5100 F1=0.5604
  class 1 (automobile): P=0.6891 R=0.8200 F1=0.7489
  class 2 (bird): P=0.4615 R=0.4800 F1=0.4706
  class 3 (cat): P=0.3429 R=0.4800 F1=0.4000
  class 4 (deer): P=0.6111 R=0.3300 F1=0.4286
  class 5 (dog): P=0.4727 R=0.5200 F1=0.4952
  class 6 (frog): P=0.6000 R=0.6900 F1=0.6419
  class 7 (horse): P=0.7111 R=0.6400 F1=0.6737
  class 8 (ship): P=0.6182 R=0.6800 F1=0.6476
  class 9 (truck): P=0.7237 R=0.5500 F1=0.6250


## 6.2: Adding convolutional layers

In [47]:
class VGG11_Add(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            # added an extra layer
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU()
        )

        # final feature map = 512×2×2 = 2048
        self.classifier = nn.Sequential(
            nn.Linear(512 * 2 * 2, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)


Preparing data after adding a layer

In [48]:
transform_cnn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset_cnn = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cnn)
testset_cnn = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cnn)

trainset_cnn = get_subset(trainset_cnn, 500)
testset_cnn = get_subset(testset_cnn, 100)

trainloader = DataLoader(trainset_cnn, batch_size=64, shuffle=True)
testloader = DataLoader(testset_cnn, batch_size=64, shuffle=False)

Training CNN after adding layer

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG11_Add(num_classes=10).to(device)

model_path = "vgg11_add.pth"

#checking if model was saved and load it
if os.path.exists('vvgg11_add.pth'):
  print("Loading saved model...")
  model.load_state_dict(torch.load(model_path, map_location=device))
  model.eval()
else:
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

  num_epochs = 10
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0
      for images, labels in trainloader:
          images, labels = images.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      #printing the loss to see that the model is training
      print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss/len(trainloader):.4f}")
  torch.save(model.state_dict(), model_path)

Epoch [1/10] - Loss: 2.1776
Epoch [2/10] - Loss: 1.7579
Epoch [3/10] - Loss: 1.4853
Epoch [4/10] - Loss: 1.2147
Epoch [5/10] - Loss: 0.9445
Epoch [6/10] - Loss: 0.6897
Epoch [7/10] - Loss: 0.6461
Epoch [8/10] - Loss: 0.4287
Epoch [9/10] - Loss: 0.3466
Epoch [10/10] - Loss: 0.1936


Evaluation CNN after adding layer

In [50]:
def evaluate_model(model, testloader, class_names):
    model.eval()
    device = next(model.parameters()).device
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    acc = accuracy_score(all_labels, all_preds)
    prec, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None)
    macro_prec, macro_rec, macro_f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')

    print("===== CNN (VGG11-Added, CIFAR-10) =====")
    print("Confusion Matrix (rows=true, cols=pred):")
    print(cm)
    print("\nOverall:")
    print(f"- Accuracy       : {acc:.4f}")
    print(f"- Macro Precision: {macro_prec:.4f}")
    print(f"- Macro Recall   : {macro_rec:.4f}")
    print(f"- Macro F1       : {macro_f1:.4f}")

    for i, name in enumerate(class_names):
        print(f"  class {i} ({name}): P={prec[i]:.4f} R={rec[i]:.4f} F1={f1[i]:.4f}")
    return all_labels, all_preds

# CIFAR-10 class labels
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

labels_cnn_add, preds_cnn_add = evaluate_model(model, testloader, class_names)

row_cnn_add    = metrics_row(labels_cnn_add, preds_cnn_add, "CNN (add)")


===== CNN (VGG11-Added, CIFAR-10) =====
Confusion Matrix (rows=true, cols=pred):
[[53  3  2  4  4  5  3  6 16  4]
 [ 3 68  0  5  1  4  2  3  5  9]
 [11  1 35 17  6 16  7  5  2  0]
 [ 2  1  3 39  1 24 14 13  0  3]
 [ 2  1 12 17 33  9  7 18  1  0]
 [ 0  0  4 23  3 56  2 12  0  0]
 [ 0  0  0 16  6  3 74  1  0  0]
 [ 0  0  0  8  4  9  0 75  0  4]
 [10  8  1  5  1  0  0  0 71  4]
 [ 2 11  1  9  0  0  5  6  9 57]]

Overall:
- Accuracy       : 0.5610
- Macro Precision: 0.5825
- Macro Recall   : 0.5610
- Macro F1       : 0.5604
  class 0 (airplane): P=0.6386 R=0.5300 F1=0.5792
  class 1 (automobile): P=0.7312 R=0.6800 F1=0.7047
  class 2 (bird): P=0.6034 R=0.3500 F1=0.4430
  class 3 (cat): P=0.2727 R=0.3900 F1=0.3210
  class 4 (deer): P=0.5593 R=0.3300 F1=0.4151
  class 5 (dog): P=0.4444 R=0.5600 F1=0.4956
  class 6 (frog): P=0.6491 R=0.7400 F1=0.6916
  class 7 (horse): P=0.5396 R=0.7500 F1=0.6276
  class 8 (ship): P=0.6827 R=0.7100 F1=0.6961
  class 9 (truck): P=0.7037 R=0.5700 F1=0.6298


## 6.3: Removing convolutional layers

In [51]:
class VGG11_Remove(nn.Module):
  def __init__(self, num_classes=10):
    super(VGG11_Remove, self).__init__()
    self.features = nn.Sequential(
      nn.Conv2d(3, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
      nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2),
      nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(),
      nn.Conv2d(256, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2, 2),
      nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
      #removed 3 layers
    )
    self.classifier = nn.Sequential(
      nn.Linear(512*4*4, 4096), nn.ReLU(), nn.Dropout(0.5),
      nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
      nn.Linear(4096, num_classes)
    )

  def forward(self, x):
    x = self.features(x)
    x = torch.flatten(x, 1)
    return self.classifier(x)

Preparing data after removing layer

In [52]:
transform_cnn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset_cnn = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cnn)
testset_cnn = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cnn)

trainset_cnn = get_subset(trainset_cnn, 500)
testset_cnn = get_subset(testset_cnn, 100)

trainloader = DataLoader(trainset_cnn, batch_size=64, shuffle=True)
testloader = DataLoader(testset_cnn, batch_size=64, shuffle=False)

Training CNN after removing layer

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG11_Remove(num_classes=10).to(device)

model_path = "vgg11_remove.pth"

#checking if model was saved and load it
if os.path.exists('vvgg11_remove.pth'):
  print("Loading saved model...")
  model.load_state_dict(torch.load(model_path, map_location=device))
  model.eval()
else:
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

  num_epochs = 10
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0
      for images, labels in trainloader:
          images, labels = images.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      #printing the loss to see that the model is training
      print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss/len(trainloader):.4f}")
  torch.save(model.state_dict(), model_path)

Epoch [1/10] - Loss: 2.1993
Epoch [2/10] - Loss: 1.8422
Epoch [3/10] - Loss: 1.5895
Epoch [4/10] - Loss: 1.4464
Epoch [5/10] - Loss: 1.3135
Epoch [6/10] - Loss: 1.1836
Epoch [7/10] - Loss: 1.0615
Epoch [8/10] - Loss: 0.9439
Epoch [9/10] - Loss: 0.8021
Epoch [10/10] - Loss: 0.6571


Evaluating CNN after removing layer

In [54]:
def evaluate_model(model, testloader, class_names):
    model.eval()
    device = next(model.parameters()).device
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    acc = accuracy_score(all_labels, all_preds)
    prec, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None)
    macro_prec, macro_rec, macro_f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')

    print("===== CNN (VGG11-Remove, CIFAR-10) =====")
    print("Confusion Matrix (rows=true, cols=pred):")
    print(cm)
    print("\nOverall:")
    print(f"- Accuracy       : {acc:.4f}")
    print(f"- Macro Precision: {macro_prec:.4f}")
    print(f"- Macro Recall   : {macro_rec:.4f}")
    print(f"- Macro F1       : {macro_f1:.4f}")

    for i, name in enumerate(class_names):
        print(f"  class {i} ({name}): P={prec[i]:.4f} R={rec[i]:.4f} F1={f1[i]:.4f}")
    return all_labels, all_preds

# CIFAR-10 class labels
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

labels_cnn_remove, preds_cnn_remove = evaluate_model(model, testloader, class_names)

row_cnn_remove = metrics_row(labels_cnn_remove, preds_cnn_remove, "CNN (remove)")


===== CNN (VGG11-Remove, CIFAR-10) =====
Confusion Matrix (rows=true, cols=pred):
[[49  7  5  2  1  0  4  5 13 14]
 [ 1 80  0  1  1  0  0  1  0 16]
 [ 9  1 33  5 25  6  5  9  3  4]
 [ 2  3  6 26 12 12 14 17  2  6]
 [ 3  2  7  2 52  4  3 24  1  2]
 [ 0  1  6 17  4 41  2 25  1  3]
 [ 0  2  1  4 15  1 66  3  0  8]
 [ 2  1  1  4  1  3  0 87  0  1]
 [ 8  9  1  1  1  0  0  1 66 13]
 [ 2  6  1  1  0  0  0  4  4 82]]

Overall:
- Accuracy       : 0.5820
- Macro Precision: 0.5869
- Macro Recall   : 0.5820
- Macro F1       : 0.5686
  class 0 (airplane): P=0.6447 R=0.4900 F1=0.5568
  class 1 (automobile): P=0.7143 R=0.8000 F1=0.7547
  class 2 (bird): P=0.5410 R=0.3300 F1=0.4099
  class 3 (cat): P=0.4127 R=0.2600 F1=0.3190
  class 4 (deer): P=0.4643 R=0.5200 F1=0.4906
  class 5 (dog): P=0.6119 R=0.4100 F1=0.4910
  class 6 (frog): P=0.7021 R=0.6600 F1=0.6804
  class 7 (horse): P=0.4943 R=0.8700 F1=0.6304
  class 8 (ship): P=0.7333 R=0.6600 F1=0.6947
  class 9 (truck): P=0.5503 R=0.8200 F1=0.6586


## 6.4: Larger kernel size

Kernel size of 5x5

In [55]:
class VGG11_kernel5(nn.Module):
  def __init__(self, num_classes=10):
    super(VGG11_kernel5, self).__init__()
    self.features = nn.Sequential(
            #Chaning the kernel size to 5x5
            nn.Conv2d(3, 64, 5, 1, 2), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 5, 1, 2), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 5, 1, 2), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 5, 1, 2), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, 5, 1, 2), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 5, 1, 2), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(512, 512, 5, 1, 2), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 5, 1, 2), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d(2, 2)
        )
    self.classifier = nn.Sequential(
            nn.Linear(512, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )

  def forward(self, x):
    x = self.features(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x

Prepare data for kernel 5x5

In [56]:
transform_cnn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


trainset_cnn = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cnn)
testset_cnn = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cnn)

trainset_cnn = get_subset(trainset_cnn, 500)
testset_cnn = get_subset(testset_cnn, 100)

trainloader = DataLoader(trainset_cnn, batch_size=64, shuffle=True)
testloader = DataLoader(testset_cnn, batch_size=64, shuffle=False)

Training CNN with 5x5 kernel size

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG11_kernel5(num_classes=10).to(device)

model_path = "vgg11_k5.pth"

#checking if model was saved and load it
if os.path.exists('vvgg11_k5.pth'):
  print("Loading saved model...")
  model.load_state_dict(torch.load(model_path, map_location=device))
  model.eval()
else:
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

  num_epochs = 10
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0
      for images, labels in trainloader:
          images, labels = images.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      #printing the loss to see that the model is training
      print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss/len(trainloader):.4f}")
  torch.save(model.state_dict(), model_path)

Epoch [1/10] - Loss: 2.0477
Epoch [2/10] - Loss: 1.6028
Epoch [3/10] - Loss: 1.3285
Epoch [4/10] - Loss: 1.1040
Epoch [5/10] - Loss: 0.8558
Epoch [6/10] - Loss: 0.6944
Epoch [7/10] - Loss: 0.6220
Epoch [8/10] - Loss: 0.4390
Epoch [9/10] - Loss: 0.2942
Epoch [10/10] - Loss: 0.4437


Evaluating the CNN with kernel size 5x5

In [58]:
def evaluate_model(model, testloader, class_names):
    model.eval()
    device = next(model.parameters()).device
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    acc = accuracy_score(all_labels, all_preds)
    prec, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None)
    macro_prec, macro_rec, macro_f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')

    print("===== CNN (VGG11_Kernel5, CIFAR-10) =====")
    print("Confusion Matrix (rows=true, cols=pred):")
    print(cm)
    print("\nOverall:")
    print(f"- Accuracy       : {acc:.4f}")
    print(f"- Macro Precision: {macro_prec:.4f}")
    print(f"- Macro Recall   : {macro_rec:.4f}")
    print(f"- Macro F1       : {macro_f1:.4f}")

    for i, name in enumerate(class_names):
        print(f"  class {i} ({name}): P={prec[i]:.4f} R={rec[i]:.4f} F1={f1[i]:.4f}")
    return all_labels, all_preds

# CIFAR-10 class labels
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

labels_cnn_k5, preds_cnn_k5 = evaluate_model(model, testloader, class_names)

row_cnn_k5      = metrics_row(labels_cnn_k5, preds_cnn_k5, "CNN (Kernel 5x5)")

===== CNN (VGG11_Kernel5, CIFAR-10) =====
Confusion Matrix (rows=true, cols=pred):
[[38 22  1  1  2  1  4  1 24  6]
 [ 0 97  0  0  0  0  0  0  2  1]
 [17  6 25  7 19 10  7  8  1  0]
 [ 3 17  5 19 14 15 20  4  0  3]
 [ 4  9  5  3 53  4  7  9  1  5]
 [ 0  6  8 11  8 44 10  9  2  2]
 [ 0 13  0  0 21  2 62  0  1  1]
 [ 2  8  1  1 11  6  2 62  1  6]
 [ 4 27  0  0  0  0  0  0 63  6]
 [ 1 51  0  0  0  0  0  1  5 42]]

Overall:
- Accuracy       : 0.5050
- Macro Precision: 0.5315
- Macro Recall   : 0.5050
- Macro F1       : 0.4898
  class 0 (airplane): P=0.5507 R=0.3800 F1=0.4497
  class 1 (automobile): P=0.3789 R=0.9700 F1=0.5449
  class 2 (bird): P=0.5556 R=0.2500 F1=0.3448
  class 3 (cat): P=0.4524 R=0.1900 F1=0.2676
  class 4 (deer): P=0.4141 R=0.5300 F1=0.4649
  class 5 (dog): P=0.5366 R=0.4400 F1=0.4835
  class 6 (frog): P=0.5536 R=0.6200 F1=0.5849
  class 7 (horse): P=0.6596 R=0.6200 F1=0.6392
  class 8 (ship): P=0.6300 R=0.6300 F1=0.6300
  class 9 (truck): P=0.5833 R=0.4200 F1=0.4884


## 6.5: Smaller kernel size

Kept the kernel size 3x3 as small, which was already implemented above

# 7: Evaluation Table

Building Evaluation Table

In [59]:
rows = [
    row_gnb_scratch,
    row_gnb_sklearn,
    row_dt_scratch,
    row_dt_sklearn,
    row_mlp_initial,
    row_mlp_deeper,
    row_mlp_shallower,
    row_mlp_wider,
    row_mlp_narrower,
    row_cnn,
    row_cnn_add,
    row_cnn_remove,
    row_cnn_k5
]

df_results = pd.DataFrame(rows)
df_results

,Model,Accuracy,Macro Precision,Macro Recall,Macro F1
0,Naive Bayes (Scratch),0.790,0.795115,0.790,0.790580
1,Scikit’s Gaussian Naive Bayes,0.790,0.795115,0.790,0.790580
2,Decision Tree (Scratch),0.599,0.609647,0.599,0.601414
3,Scikit’s implementation of a Decision Tree,0.602,0.612927,0.602,0.604390
4,MLP,0.827,0.829060,0.827,0.827116
5,"MLP (Deeper, PCA-50)",0.830,0.832858,0.830,0.830556
6,"MLP (Shallower, PCA-50)",0.813,0.814139,0.813,0.812426
7,"MLP (Wider, PCA-50)",0.827,0.830339,0.827,0.827329
8,"MLP (Narrower, PCA-50)",0.810,0.811288,0.810,0.809018
9,CNN (VGG11),0.570,0.585224,0.570,0.569186
